# Basic Process

*Not all the code is shown below; just the highlights.*

## scrape.py

### 1. Load station data

```python
def get_fares_from_google_maps():
    fares_waiting: set = load_fares_waiting(TARGET_STATIONS)
    fare_count: int = file_counter(FARES)                                                                   
    if DEST in fares_waiting:
        fares_waiting.remove(DEST)
    stations: List[Text] = list(fares_waiting)                      
    stations.sort()    
```

### 2. Input the destination station

``` python
    wait(3)  # manually switch desktop
    goto_search_input()
    enter_text(DEST)
    wait(3)
    locate_directions_arrow()
    wait(2)
```

### 3. Iterate through all the starting stations

```python
    # input the station starting points
    while len(stations) > 0:
        errors = 0
        station = stations.pop(0)
        st_name = f"{station} Station"
        change_starting_station(st_name) 

        if errors > 20:
            LOG.debug("Too many errors. Quitting...")
            exit()
        if station not in load_stations(ISSUE_STATIONS):
            if capture_fare(f"{FARES}{st_name}_{DEST}.png"):
                fare_count += 1
            else:
                LOG.debug(f"Fare not found: {st_name} to {DEST}")
                with open(ISSUE_STATIONS, "a+") as f:
                    f.write(station+"\n")
                errors += 1
```

`capture_fare()` takes a screenshot of the fare and extracts the price with pytesseract.  
It is loaded from `googlemaps.py`.

## googlemaps.py

```python
def capture_fare(saveto: Text) -> bool:
    """Screenshot the first fare price. Returns bool."""
    attempts = 0
    max_attempts = 5
    while attempts < max_attempts:
        try:
            yenX, yenY = pyautogui.locateCenterOnScreen(YEN,
                    region=(50, 460, 80, 50), confidence=0.9)
            pyautogui.moveTo(yenX, yenY)
            img = pyautogui.screenshot(
                region=(yenX-10, yenY-10, 60, 20))
            img.save(saveto)
            actual_fare = image_to_fare(img)  # pytesseract
            LOG.debug(f"{actual_fare}, {saveto}")
            return True
        except TypeError:
            LOG.debug("Yen symbol not found. Skipping...")
        except:
            LOG.debug("Unknown error with capture_fare(). Quitting...")
            attempts += max_attempts
            return False
        wait(2)
        attempts += 1
```

`capture_fare()` saves an image of the fare, and converts the image to a value with `image_to_fare()`.  
They are saved in `icons/` and `results/`.

## Examples

`icons/` data is in `png` format and looks like this;  
<img src="./icons/Kiba Station_Shinjuku Station.png" />
![alt text]("icons/Kiba Station_Shinjuku Station.png")  
[icons/]<"Kiba Station_Shinjuku Station.png">

`results/` data is in `txt` format and looks like this;  
```bash
280_Shinjuku Station_Kayabacho  
220_Shinjuku Station_Kinshicho  
210_Shinjuku Station_Kita-Sando  
```